In [1]:
import os

In [2]:
os.environ["XLNX_ENABLE_FINGERPRINT_CHECK"] = "0"

In [3]:
%pwd

'/root/jupyter_notebooks'

In [4]:
cd /home/ubuntu/workspace/builded_models

/home/ubuntu/workspace/builded_models


In [5]:
from pynq_dpu import DpuOverlay
overlay = DpuOverlay("dpu.bit")

In [6]:
ls

Untitled-1.ipynb                  resnet18_pytorch_deneme.ipynb
archive/                          target_kv260/
kv260_train_resnet18_vcor.xmodel  target_kv260.tar
resnet18_pytorch.ipynb


In [7]:
overlay.load_model("kv260_train_resnet18_vcor.xmodel")

In [8]:
%pwd

'/home/ubuntu/workspace/builded_models'

In [9]:
import os
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
from random import seed, shuffle
%matplotlib inline

In [10]:
# Global variables
IMG_W = 224
IMG_H = 224
#MAX_NUM_TEST_IMAGES = 1556

labelNames_dict = {
    "beige": 0, "black": 1, "blue": 2, "brown": 3, "gold": 4,
    "green": 5, "grey": 6, "orange": 7, "pink": 8, "purple": 9,
    "red": 10, "silver": 11, "tan": 12, "white": 13, "yellow": 14
}

labelNames_list = [
    "beige", "black", "blue", "brown", "gold",
    "green", "grey", "orange", "pink", "purple",
    "red", "silver", "tan", "white", "yellow"
]

# Define preprocessing functions
_MEAN = [123.68, 116.78, 103.94]  # RGB mean values

In [11]:
# Define preprocessing functions
_MEAN = [123.68, 116.78, 103.94]  # RGB mean values

def resize_shortest_edge(image, size):
    H, W = image.shape[:2]
    if H >= W:
        nW = size
        nH = int(float(H)/W * size)
    else:
        nH = size
        nW = int(float(W)/H * size)
    return cv2.resize(image, (nW, nH))

def mean_image_subtraction(image, means):
    B, G, R = cv2.split(image)
    B = B - means[0]
    G = G - means[1]
    R = R - means[2]
    image = cv2.merge([R, G, B])
    return image

def central_crop(image, crop_height, crop_width):
    image_height = image.shape[0]
    image_width = image.shape[1]
    offset_height = (image_height - crop_height) // 2
    offset_width = (image_width - crop_width) // 2
    return image[offset_height:offset_height + crop_height, offset_width:offset_width + crop_width, :]

def preprocess_fn(image, crop_height=224, crop_width=224):
    image = resize_shortest_edge(image, 256)
    image = mean_image_subtraction(image, _MEAN)
    image = central_crop(image, crop_height, crop_width)
    return image

In [12]:
# Softmax and label prediction functions
def calculate_softmax(data):
    result = np.exp(data - np.max(data))
    return result / np.sum(result)

def predict_label(softmax):
    return labelNames_list[np.argmax(softmax)]


In [13]:
# Clear old processed files
def clear_processed_files(image_folder):
    for category in os.listdir(image_folder):
        category_path = os.path.join(image_folder, category)
        if os.path.isdir(category_path):
            for file_name in os.listdir(category_path):
                if '_' in file_name and file_name.endswith('.jpg'):
                    file_path = os.path.join(category_path, file_name)
                    os.remove(file_path)
                   # print(f"Removed old processed file: {file_path}")

In [14]:
%pwd

'/home/ubuntu/workspace/builded_models'

In [15]:
cd archive

/home/ubuntu/workspace/builded_models/archive


In [16]:
image_folder = 'test'  
clear_processed_files(image_folder)

In [17]:
# Prepare the image dataset
print("Dataset directory:", image_folder)
categories = sorted([d for d in os.listdir(image_folder) if os.path.isdir(os.path.join(image_folder, d))])
print("Categories found:", categories)
category_to_label = {category: idx for idx, category in enumerate(categories)}

# Get list of image files and their labels
image_files = []
image_labels = []

for category in categories:
    category_path = os.path.join(image_folder, category)
    #print("Processing category:", category, "Path:", category_path)
    category_files = os.listdir(category_path)
    #print("Files in category:", category_files)
    for image_file in os.listdir(category_path):
          if image_file.endswith('.jpg') and not '_' in image_file:
            image_files.append(os.path.join(category_path, image_file))
            image_labels.append(category_to_label[category])

print("Total images found:", len(image_files))

# Shuffle images for randomness
#seed(1)
#shuffle(image_files)

# Process images to ensure they are resized and normalized
processed_image_files = []
counter = np.zeros(len(labelNames_list), dtype="uint32")

for img_file in image_files:
   # print("Processing image file:", img_file)
    img_orig = cv2.imread(img_file)
    if img_orig is None:
        print("Failed to read image:", img_file)
        continue  # Skip files that are not images or cannot be read

    classname = img_file.split("/")[-2]
    label = labelNames_dict[classname]
    new_img_file = img_file.split(".")[0] + "_" + classname + ".jpg"
    
    img_resiz = cv2.resize(img_orig, (IMG_W, IMG_H))
    cv2.imwrite(new_img_file, img_resiz)
    
    processed_image_files.append(new_img_file)
    counter[label] += 1

print("Classes histogram in test folder: ", counter)
print("Number of images in test folder = ", len(processed_image_files))


Dataset directory: test
Categories found: ['beige', 'black', 'blue', 'brown', 'gold', 'green', 'grey', 'orange', 'pink', 'purple', 'red', 'silver', 'tan', 'white', 'yellow']
Total images found: 1556
Classes histogram in test folder:  [ 90  87 159 121  45 121  92 114 103 115 136  77  86  86 124]
Number of images in test folder =  1556


In [18]:
# Initialize DPU runner
dpu = overlay.runner

inputTensors = dpu.get_input_tensors()
outputTensors = dpu.get_output_tensors()

shapeIn = tuple(inputTensors[0].dims)
shapeOut = tuple(outputTensors[0].dims)
outputSize = int(outputTensors[0].get_data_size() / shapeIn[0])

print(f"Input tensor shape: {shapeIn}")
print(f"Output tensor shape: {shapeOut}")

input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
output_data = [np.empty(shapeOut, dtype=np.float32, order="C")]

print(f"Initial memory usage (input_data): {input_data[0].nbytes / (1024 ** 2)} MB")
print(f"Initial memory usage (output_data): {output_data[0].nbytes / (1024 ** 2)} MB")



Input tensor shape: (1, 224, 224, 3)
Output tensor shape: (1, 15)
Initial memory usage (input_data): 0.57421875 MB
Initial memory usage (output_data): 5.7220458984375e-05 MB


In [19]:
# Function to run DPU inference
def run_inference(image):
    try:
        if(image.shape == (shapeIn[1], shapeIn[2], shapeIn[3])):
            input_data[0][0, ...] = (image /255.0).astype(np.float32)
            print("--")
        else:
            exit(-1)
        #print(f"Preprocessed image shape: {preprocessed.shape}")
        #print(f"Preprocessed image dtype: {preprocessed.dtype}")
        print("Running inference...")
        job_id = dpu.execute_async(input_data, output_data)
        dpu.wait(job_id)
        print("Inference completed...")
        softmax = calculate_softmax(output_data[0][0])
        return predict_label(softmax)
    except Exception as e:
        print("An error occurred during inference:", e)
        return None


In [20]:
# Load processed images and their labels
processed_images = [(cv2.imread(img_file), img_file.split("_")[-1].split(".")[0]) for img_file in processed_image_files]

# Ensure the images are correctly read
processed_images = [(img, label) for img, label in processed_images if img is not None]

In [21]:
processed_images 

[(array([[[209, 199, 175],
          [210, 200, 176],
          [209, 198, 171],
          ...,
          [177, 151, 115],
          [173, 147, 111],
          [173, 147, 111]],
  
         [[210, 200, 176],
          [212, 203, 176],
          [212, 201, 174],
          ...,
          [174, 148, 112],
          [174, 148, 112],
          [174, 148, 112]],
  
         [[210, 201, 174],
          [212, 203, 176],
          [214, 203, 176],
          ...,
          [172, 145, 111],
          [173, 146, 112],
          [173, 146, 112]],
  
         ...,
  
         [[ 86,  91,  89],
          [ 86,  91,  89],
          [ 85,  90,  88],
          ...,
          [130, 164, 180],
          [130, 164, 180],
          [131, 165, 181]],
  
         [[ 87,  92,  90],
          [ 86,  91,  89],
          [ 85,  90,  88],
          ...,
          [128, 162, 178],
          [124, 160, 176],
          [124, 160, 176]],
  
         [[ 87,  92,  90],
          [ 86,  91,  89],
          [ 85,  90,  88

In [22]:
#processed_images = [(cv2.imread(img_file), img_file.split("_")[-1].split(".")[0]) for img_file in processed_image_files]

In [23]:
#image = processed_images[0][0].shape

In [24]:
#predicted_label = run_inference(image)

In [25]:
# Run inference and calculate accuracy
correct_predictions = 0
total_predictions = len(processed_images)

start_time = time.time()

for image, true_label in processed_images:
    print(f"Processing image with true label: {true_label}")
    predicted_label = run_inference(image)
    print(f"Predicted label: {predicted_label}")
    if predicted_label == true_label:
        correct_predictions += 1

end_time = time.time()

accuracy = (correct_predictions / total_predictions) * 100
fps = total_predictions / (end_time - start_time)

print(f"Accuracy: {accuracy:.2f}%")
print(f"Performance: {fps:.2f} FPS")


Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: beige
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: beige
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: white
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: white
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: silver
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: silver
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: brown
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: silver
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: blue
Processi

Inference completed...
Predicted label: beige
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: green
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: beige
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: grey
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: grey
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: brown
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: blue
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: white
Processing image with true label: beige
--
Running inference...
Inference completed...
Predicted label: silver
Processing image with true label: beige
--
Running inference...
Inference co